In [6]:
import csv

def skroc_plik_csv(nazwa_pliku, liczba_wierszy):
    with open(nazwa_pliku, 'r') as plik_wejsciowy:
        czytnik_csv = csv.reader(plik_wejsciowy)
        naglowek = next(czytnik_csv)  # Pomijanie nagłówka

        dane = []
        for wiersz in czytnik_csv:
            dane.append(wiersz)
            if len(dane) >= liczba_wierszy:
                break

    with open(nazwa_pliku, 'w', newline='') as plik_wyjsciowy:
        pisarz_csv = csv.writer(plik_wyjsciowy)
        pisarz_csv.writerow(naglowek)
        pisarz_csv.writerows(dane)

    print(f"Plik {nazwa_pliku} został skrócony do {liczba_wierszy} wierszy.")


In [7]:
nazwa_pliku_csv = 'taxi.csv'
liczba_wierszy = 400000  # Określona liczba wierszy, do której chcesz skrócić plik

skroc_plik_csv(nazwa_pliku_csv, liczba_wierszy)

Plik taxi.csv został skrócony do 400000 wierszy.
